  - https://deepsense.ai/region-of-interest-pooling-explained/

  - https://medium.com/@fractaldle/guide-to-build-faster-rcnn-in-pytorch-95b10c273439
  
Fast R-CNN used ROI pooling to extract features for each and every proposal suggested by selective search (Fast RCNN) or Region Proposal network (RPN in Faster R- CNN). We will see how this ROI pooling works and later pass the rpn proposals which we have computed in section 4 to this layer. Further we will see how this layer is connected to a classification and regression layer to compute the class probabilities and bounding boxes coordinates respectively.

Region of interest pooling (also known as RoI pooling) purpose is to perform max pooling on inputs of non-uniform sizes to obtain fixed-size feature maps (e.g. 7×7). This layer takes two inputs
  - A fixed-size feature map obtained from a deep convolutional network with several convolutions and max-pooling layers
  - An Nx5 matrix of representing a list of regions of interest, where N is the number of RoIs. The first column represents the image index and the remaining four are the co-ordinates of the top left and bottom right corners of the region.

What does the RoI pooling actually do? For every region of interest from the input list, it takes a section of the input feature map that corresponds to it and scales it to some pre-defined size (e.g., 7×7). The scaling is done by:

  - Dividing the region proposal into equal-sized sections (the number of which is the same as the dimension of the output)
  - Finding the largest value in each section
  - Copying these max values to the output buffer

The result is that from a list of rectangles with different sizes we can quickly get a list of corresponding feature maps with a fixed size. Note that the dimension of the RoI pooling output doesn’t actually depend on the size of the input feature map nor on the size of the region proposals. It’s determined solely by the number of sections we divide the proposal into. What’s the benefit of RoI pooling? One of them is processing speed. If there are multiple object proposals on the frame (and usually there’ll be a lot of them), we can still use the same input feature map for all of them. Since computing the convolutions at early stages of processing is very expensive, this approach can save us a lot of time. The diagram below shows the working of ROI pooling.
ROI Pooling 2x2

  - https://discuss.pytorch.org/t/pytorch-roi-pooling-implementation-deviation/21507

In [ ]:
F.adaptive_max_pool2d

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def roi_pooling(feature_map, rois, size=(8, 8)): # code de base 7,7
    """
    :param feature_map: (1, C, H, W)
    :param rois: (1, N, 4) N refers to bbox num, 4 represent (ltx, lty, w, h) 
    :param size: output size
    :return: (1, C, size[0], size[1])
    """
    output = []
    rois_num = rois.size(1)

    for i in range(rois_num):
        roi = rois[0][i]
        x, y, w, h = roi
        output.append(F.adaptive_max_pool2d(feature_map[:, :, y:y+h, x:x+w], size))
        # 1er : taille du batch , 2eme : nb canaux

    return torch.cat(output)


if __name__ == '__main__':
    test_tensor = torch.tensor([
        [0.88, 0.44, 0.14, 0.16, 0.37, 0.77, 0.96, 0.27],
        [0.19, 0.45, 0.57, 0.16, 0.63, 0.29, 0.71, 0.70],
        [0.66, 0.26, 0.82, 0.64, 0.54, 0.73, 0.59, 0.26],
        [0.85, 0.34, 0.76, 0.84, 0.29, 0.75, 0.62, 0.25],
        [0.32, 0.74, 0.21, 0.39, 0.34, 0.03, 0.33, 0.48],
        [0.20, 0.14, 0.16, 0.13, 0.73, 0.65, 0.96, 0.32],
        [0.19, 0.69, 0.09, 0.86, 0.88, 0.07, 0.01, 0.48],
        [0.83, 0.24, 0.97, 0.04, 0.24, 0.35, 0.50, 0.91]
    ])
    test_tensor = test_tensor.view(1, 1, 8, 8)
    rois = torch.tensor([[0,1,8,8]])  # colonne ligne, colonne ligne (exclu)
    rois = torch.tensor([[0,4,8,8]])
    # rois = torch.tensor([[1,4,7,8]])
    # rois = torch.tensor([[4,1,8,7]])
    #rois = torch.tensor([[0, 3, 7, 5]]) # original
    print("rois", rois)
    rois = rois.view(1, -1, 4)
    print("rois2", rois)
    output = roi_pooling(test_tensor, rois, (2, 2))
    print(output)

rois tensor([[0, 4, 8, 8]])
rois2 tensor([[[0, 4, 8, 8]]])
tensor([[[[0.7400, 0.9600],
          [0.9700, 0.9100]]]])


In [ ]:
import torch
import torch.nn as nn

def roi_pooling(feature_map, rois, size=(8, 8)): # code de base 7,7
    """
    :param feature_map: (1, C, H, W)
    :param rois: (1, N, 4) N refers to bbox num, 4 represent (ltx, lty, w, h) 
    :param size: output size
    :return: (1, C, size[0], size[1])
    """
    output = []
    rois_num = rois.size(1)
    m = nn.AdaptativeMaxPool2d((2,2))

    for i in range(rois_num):
        roi = rois[0][i]
        x, y, w, h = roi
        output.append()

    return torch.cat(output)


if __name__ == '__main__':
    test_tensor = torch.tensor([
        [0.88, 0.44, 0.14, 0.16, 0.37, 0.77, 0.96, 0.27],
        [0.19, 0.45, 0.57, 0.16, 0.63, 0.29, 0.71, 0.70],
        [0.66, 0.26, 0.82, 0.64, 0.54, 0.73, 0.59, 0.26],
        [0.85, 0.34, 0.76, 0.84, 0.29, 0.75, 0.62, 0.25],
        [0.32, 0.74, 0.21, 0.39, 0.34, 0.03, 0.33, 0.48],
        [0.20, 0.14, 0.16, 0.13, 0.73, 0.65, 0.96, 0.32],
        [0.19, 0.69, 0.09, 0.86, 0.88, 0.07, 0.01, 0.48],
        [0.83, 0.24, 0.97, 0.04, 0.24, 0.35, 0.50, 0.91]
    ])
    test_tensor = test_tensor.view(1, 1, 8, 8)
    # rois = torch.tensor([[3,0,7,6]])
    # rois = torch.tensor([[0,3,6,7]])
    # rois = torch.tensor([[1,4,7,8]])
    # rois = torch.tensor([[4,1,8,7]])
    rois = torch.tensor([[0, 3, 7, 5]]) # original
    print("rois", rois)
    rois = rois.view(1, -1, 4)
    print("rois2", rois)
    output = roi_pooling(test_tensor, rois, (2, 2))
    print(output)

In [17]:
import torch

input = torch.randn(2, 5, 4, 3)

print(input)

tensor([[[[ 0.8074, -0.3371, -0.8654],
          [-0.1460, -1.3254,  1.3573],
          [ 0.6213, -1.8528,  2.5462],
          [-0.4008,  0.8328, -1.5651]],

         [[ 1.2837, -0.8033, -0.2083],
          [ 0.3377, -0.4712, -1.0685],
          [-0.1549,  0.0553, -0.2817],
          [-0.0830,  0.8057,  0.9261]],

         [[ 0.4635, -0.0745,  0.2781],
          [-0.3692, -0.6549, -1.1227],
          [-0.8927, -1.6569, -1.1249],
          [ 0.6579,  1.3051,  0.3052]],

         [[-0.6215, -2.0589,  1.0627],
          [-0.1711,  1.3006,  1.2464],
          [ 0.0067, -0.5315,  2.2976],
          [-1.0205, -1.7962, -0.1501]],

         [[ 0.3515, -1.1050, -0.2083],
          [ 0.8956, -1.2435,  0.4527],
          [ 0.9905, -1.5784, -0.6846],
          [-0.0431,  1.8200, -1.5333]]],


        [[[ 2.0422, -1.1704, -0.2396],
          [-0.5926, -0.5615,  0.7232],
          [ 1.7881,  0.8058,  0.6025],
          [ 0.7017, -0.1684,  0.0901]],

         [[-0.5738, -0.9466,  0.2282],
          [

In [ ]:
    test_tensor = torch.tensor([
        [0.88, 0.44, 0.14, 0.16, 0.37, 0.77, 0.96, 0.27],
        [0.19, 0.45, 0.57, 0.16, 0.63, 0.29, 0.71, 0.70],
        [0.66, 0.26, 0.82, 0.64, 0.54, 0.73, 0.59, 0.26],
        [0.85, 0.34, 0.76, 0.84, 0.29, 0.75, 0.62, 0.25],
        [0.32, 0.74, 0.21, 0.39, 0.34, 0.03, 0.33, 0.48],
        [0.20, 0.14, 0.16, 0.13, 0.73, 0.65, 0.96, 0.32],
        [0.19, 0.69, 0.09, 0.86, 0.88, 0.07, 0.01, 0.48],
        [0.83, 0.24, 0.97, 0.04, 0.24, 0.35, 0.50, 0.91]
        
        print